In [ ]:
from dask.distributed import Client, progress

In [ ]:
client = Client(threads_per_worker=4, n_worker=1)
client

In [ ]:
import time
import random

In [ ]:
def inc(x):
    time.sleep(random.random())
    return x + 1

In [ ]:
def double(x):
    time.sleep(random.random())
    return 2 * x

In [ ]:
def add(x,y):
    time.sleep(random.random())
    return x + y

In [ ]:
inc(1)

In [ ]:
future = client.submit(inc, 1)
future

In [ ]:
future.result()

In [ ]:
zs = []

In [ ]:
%%time

for i in range(256):
    x = client.submit(inc, i)    
    y = client.submit(double, x) 
    z = client.submit(add, x, y)
    zs.append(z)

In [ ]:
total = client.submit(sum,zs)

In [ ]:
client.cluster.scale(10)

In [ ]:
L = zs
while len(L) > 1:
    new_L = []
    for i in range(0, len(L), 2):
        future = client.submit(add, L[i], L[i + 1])  # add neighbors
        new_L.append(future)
    L = new_L                                   # swap old list for new

In [ ]:
del future, L, new_L, total  # clear out some old work


In [ ]:
from dask.distributed import as_completed

zs = client.map(inc, zs)
seq = as_completed(zs)

while seq.count() > 1:  # at least two futures left
    a = next(seq)
    b = next(seq)
    new = client.submit(add, a, b)  # add them together
    seq.add(new)                    # add new future back into loop